In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [54]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import cube as cube_lib
import solver as solver_lib

In [4]:
model = tf.keras.models.load_model('./model')

In [10]:
evaluation = solver_lib.evaluate_solver(
    lambda cube: solver_lib.GreedySolver(cube, model, 2),
    trajectory_length=8,
    max_num_steps=8,
    num_trials=10)

evaluation

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame()

for greedy_depth in [1, 2, 3]:
    for trajectory_length in [5, 8, 11, 14, 17, 21]:
        eval_df = solver_lib.evaluate_solver(
            lambda cube: solver_lib.GreedySolver(cube, model, greedy_depth),
            trajectory_length=trajectory_length,
            max_num_steps=trajectory_length+2,
            num_trials=50)
        eval_df['greedy_depth'] = greedy_depth
        df = df.append(eval_df, ignore_index=True)
       

df

In [ ]:
gr = df.groupby(['num_steps_scrambled', 'greedy_depth']).agg(
    mean_steps_to_solve=pd.NamedAgg(column='num_steps_to_solve', aggfunc='mean'),
    prob_solved=pd.NamedAgg(column='solved', aggfunc='mean'))

gr

In [ ]:
gr.prob_solved.unstack().plot()
_ = plt.ylabel('Probability to solve')

In [ ]:
gr.mean_steps_to_solve.unstack().plot()
_ = plt.ylabel('Mean steps to solve (when successful)')